# pyspark_ex(2) 

In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1534132306360_0012,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
<SparkContext master=yarn appName=remotesparkmagics>

## csv파일 불러올 떄 sc.textFile로 오면 RDD형태로 로드된다.
- printSchema()와 show()할 때 안됨

In [2]:
data = sc.textFile("/HdiSamples/HdiSamples/SensorSampleData/building/building.csv")

In [6]:
csv = sc.textFile("/HdiSamples/HdiSamples/SensorSampleData/building/building.csv")
csv.printSchema()
csv.show(truncate=False)

'RDD' object has no attribute 'printSchema'
Traceback (most recent call last):
AttributeError: 'RDD' object has no attribute 'printSchema'



## csv파일 불러올 떄 spark.read.csv()로 하면 df로 로드됨.
- printSchema()와 show()할 때 됨

In [7]:
building_csv = spark.read.csv("/HdiSamples/HdiSamples/SensorSampleData/building/building.csv", header=True, inferSchema=True)
building_csv.printSchema()
building_csv.show()

root
 |-- BuildingID: integer (nullable = true)
 |-- BuildingMgr: string (nullable = true)
 |-- BuildingAge: integer (nullable = true)
 |-- HVACproduct: string (nullable = true)
 |-- Country: string (nullable = true)

+----------+-----------+-----------+-----------+------------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|
+----------+-----------+-----------+-----------+------------+
|         1|         M1|         25|     AC1000|         USA|
|         2|         M2|         27|     FN39TG|      France|
|         3|         M3|         28|     JDNS77|      Brazil|
|         4|         M4|         17|     GG1919|     Finland|
|         5|         M5|          3|    ACMAX22|   Hong Kong|
|         6|         M6|          9|     AC1000|   Singapore|
|         7|         M7|         13|     FN39TG|South Africa|
|         8|         M8|         25|     JDNS77|   Australia|
|         9|         M9|         11|     GG1919|      Mexico|
|        10|        M10|         23|  

In [13]:
building_csv

DataFrame[BuildingID: int, BuildingMgr: string, BuildingAge: int, HVACproduct: string, Country: string]

In [17]:
b_data = building_csv.select("BuildingID", "BuildingAge", "HVACproduct")
b_data.show()

+----------+-----------+-----------+
|BuildingID|BuildingAge|HVACproduct|
+----------+-----------+-----------+
|         1|         25|     AC1000|
|         2|         27|     FN39TG|
|         3|         28|     JDNS77|
|         4|         17|     GG1919|
|         5|          3|    ACMAX22|
|         6|          9|     AC1000|
|         7|         13|     FN39TG|
|         8|         25|     JDNS77|
|         9|         11|     GG1919|
|        10|         23|    ACMAX22|
|        11|         14|     AC1000|
|        12|         26|     FN39TG|
|        13|         25|     JDNS77|
|        14|         17|     GG1919|
|        15|         19|    ACMAX22|
|        16|         23|     AC1000|
|        17|         11|     FN39TG|
|        18|         25|     JDNS77|
|        19|         14|     GG1919|
|        20|         19|    ACMAX22|
+----------+-----------+-----------+

In [18]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

## HVAC데이터 스키마 지정하기

In [23]:
# load HVAC.csv
schma = StructType([
        StructField("Date", StringType(), False),
        StructField("Time", StringType(), False),
        StructField("TargetTemp", IntegerType(), False),
        StructField("ActualTemp", StringType(), False),
        StructField("System", IntegerType(), False),
        StructField("SystemAge", IntegerType(), False),
        StructField("BuildingID", IntegerType(), False),
        
    ])

## HVAC데이터 로드하고 만들어준 schema 적용

In [20]:
hvac_csv = spark.read.csv("/HdiSamples/HdiSamples/SensorSampleData/hvac/HVAC.csv", schema=schma, header=True)
hvac_csv.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- TargetTemp: integer (nullable = true)
 |-- ActualTemp: string (nullable = true)
 |-- System: integer (nullable = true)
 |-- SystemAge: integer (nullable = true)
 |-- BuildingID: integer (nullable = true)

In [32]:
hvac_csv.select('Date', 'Time', 'TargetTemp', 'ActualTemp', 'System', 'SystemAge', 'BuildingID').show()

+-------+--------+----------+----------+------+---------+----------+
|   Date|    Time|TargetTemp|ActualTemp|System|SystemAge|BuildingID|
+-------+--------+----------+----------+------+---------+----------+
| 6/1/13| 0:00:01|        66|        58|    13|       20|         4|
| 6/2/13| 1:00:01|        69|        68|     3|       20|        17|
| 6/3/13| 2:00:01|        70|        73|    17|       20|        18|
| 6/4/13| 3:00:01|        67|        63|     2|       23|        15|
| 6/5/13| 4:00:01|        68|        74|    16|        9|         3|
| 6/6/13| 5:00:01|        67|        56|    13|       28|         4|
| 6/7/13| 6:00:01|        70|        58|    12|       24|         2|
| 6/8/13| 7:00:01|        70|        73|    20|       26|        16|
| 6/9/13| 8:00:01|        66|        69|    16|        9|         9|
|6/10/13| 9:00:01|        65|        57|     6|        5|        12|
|6/11/13|10:00:01|        67|        70|    10|       17|        15|
|6/12/13|11:00:01|        69|     

## hvac_data를 .select()로 실제온도가 더 높았던 빌딩을 추출

In [12]:
hvac_data = hvac_csv.select("BuildingID", "ActualTemp", "TargetTemp").filter(col("ActualTemp") > col("TargetTemp"))
hvac_data.show()

+----------+----------+----------+
|BuildingID|ActualTemp|TargetTemp|
+----------+----------+----------+
|        18|        73|        70|
|         3|        74|        68|
|        16|        73|        70|
|         9|        69|        66|
|        15|        70|        67|
|        15|        73|        69|
|        17|        72|        69|
|         9|        69|        66|
|        18|        71|        70|
|         8|        71|        67|
|        13|        77|        66|
|         9|        69|        67|
|        10|        78|        70|
|        19|        72|        67|
|         2|        73|        67|
|        10|        75|        65|
|         2|        68|        67|
|         6|        77|        65|
|         3|        67|        66|
|         1|        68|        67|
+----------+----------+----------+
only showing top 20 rows

## 실제 온도가 더 높은 빌딩을 hot_building이라는 변수에  building_data와 join하여 넣기

In [14]:
# join
hot_buildings = building_data.join(hvac_data, "BuildingID")
hot_buildings.show()

+----------+-----------+-----------+----------+----------+
|BuildingID|BuildingAge|HVACproduct|ActualTemp|TargetTemp|
+----------+-----------+-----------+----------+----------+
|        18|         25|     JDNS77|        73|        70|
|         3|         28|     JDNS77|        74|        68|
|        16|         23|     AC1000|        73|        70|
|         9|         11|     GG1919|        69|        66|
|        15|         19|    ACMAX22|        70|        67|
|        15|         19|    ACMAX22|        73|        69|
|        17|         11|     FN39TG|        72|        69|
|         9|         11|     GG1919|        69|        66|
|        18|         25|     JDNS77|        71|        70|
|         8|         25|     JDNS77|        71|        67|
|        13|         25|     JDNS77|        77|        66|
|         9|         11|     GG1919|        69|        67|
|        10|         23|    ACMAX22|        78|        70|
|        19|         14|     GG1919|        72|        6

## hot_buildings 변수를 createOrReplaceTempView로 변환하여 출력

In [33]:
# spark SQL
hot_buildings.createOrReplaceTempView("tmpHotBuildings")
spark.sql("SELECT HVACproduct, COUNT(*) AS installations FROM tmpHotBuildings GROUP BY HVACproduct").show()


+-----------+-------------+
|HVACproduct|installations|
+-----------+-------------+
|    ACMAX22|          790|
|     AC1000|          773|
|     JDNS77|          778|
|     FN39TG|          833|
|     GG1919|          740|
+-----------+-------------+

In [16]:

# save table
building_csv.write.saveAsTable("building")
building_df = spark.sql("SELECT * FROM building")
building_df.show()

+----------+-----------+-----------+-----------+------------+
|BuildingID|BuildingMgr|BuildingAge|HVACproduct|     Country|
+----------+-----------+-----------+-----------+------------+
|         1|         M1|         25|     AC1000|         USA|
|         2|         M2|         27|     FN39TG|      France|
|         3|         M3|         28|     JDNS77|      Brazil|
|         4|         M4|         17|     GG1919|     Finland|
|         5|         M5|          3|    ACMAX22|   Hong Kong|
|         6|         M6|          9|     AC1000|   Singapore|
|         7|         M7|         13|     FN39TG|South Africa|
|         8|         M8|         25|     JDNS77|   Australia|
|         9|         M9|         11|     GG1919|      Mexico|
|        10|        M10|         23|    ACMAX22|       China|
|        11|        M11|         14|     AC1000|     Belgium|
|        12|        M12|         26|     FN39TG|     Finland|
|        13|        M13|         25|     JDNS77|Saudi Arabia|
|       

In [36]:
# Query Hive Table
spark.sql("SHOW TABLES").show()

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
| default|       building|      false|
| default|hivesampletable|      false|
| default|           hvac|      false|
|        |tmphotbuildings|       true|
+--------+---------------+-----------+